In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="68TRJG3oL0xut9Ks0kkD")
project = rf.workspace("ship-xallx").project("test-au9yp")
version = project.version(5)
dataset = version.download("yolov8")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to test-5 in yolov8:: 100%|██████████| 11019/11019 [00:01<00:00, 7156.52it/s]


In [ ]:
from google.colab import files
uploaded = files.upload()  # 로컬에서 pth파일 선택해 업로드
# 업로드가 완료되면 업로드된 파일명을 확인하고, 해당 파일명으로 모델 로드
model.load_state_dict(torch.load('best_vit_model.pth', map_location=torch.device('cpu')))
model.eval()


In [ ]:
import os

# 이미지 파일들이 있는 디렉토리 경로를 설정해주세요.
directory_path = "/content/test-5/test"

for filename in os.listdir(directory_path):
    # 파일명이 17-7-fish로 시작하는지 확인
    if filename.startswith("17-7-fish"):
        print(filename)


In [ ]:
from torch.utils.data import Dataset, DataLoader, random_split
import torch
from torch import nn
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from transformers import ViTForImageClassification, AdamW
from sklearn.metrics import accuracy_score, classification_report
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
from collections import Counter
from PIL import Image
import os
import matplotlib.pyplot as plt
import shutil
import random


In [ ]:
# 기존 폴더 경로
base_path = "/content/test-5"
folders = ["train", "valid", "test"]

# 합쳐질 임시 폴더 생성
combined_images_path = os.path.join(base_path, "combined_images")
combined_labels_path = os.path.join(base_path, "combined_labels")
os.makedirs(combined_images_path, exist_ok=True)
os.makedirs(combined_labels_path, exist_ok=True)

# 이미지 및 라벨 파일 합치기
for folder in folders:
    images_path = os.path.join(base_path, folder, "images")
    labels_path = os.path.join(base_path, folder, "labels")

    if os.path.exists(images_path):
        for image_file in os.listdir(images_path):
            # 이미지 파일 복사
            shutil.copy(os.path.join(images_path, image_file), combined_images_path)

            # 라벨 파일 매칭 및 복사
            label_file = os.path.splitext(image_file)[0] + ".txt"  # 확장자를 .txt로 변경
            if os.path.exists(os.path.join(labels_path, label_file)):
                shutil.copy(os.path.join(labels_path, label_file), combined_labels_path)

# 합친 데이터 확인
all_images = os.listdir(combined_images_path)
all_labels = os.listdir(combined_labels_path)

print(f"combined_images 폴더의 이미지 개수: {len(all_images)}개")
print(f"combined_labels 폴더의 라벨 개수: {len(all_labels)}개")

combined_images 폴더의 이미지 개수: 5504개
combined_labels 폴더의 라벨 개수: 5504개


In [ ]:
# 새로운 train/valid/test 폴더 경로 설정
new_folders = {
    "train": {"images": "/content/new_data/train/images", "labels": "/content/new_data/train/labels"},
    "valid": {"images": "/content/new_data/valid/images", "labels": "/content/new_data/valid/labels"},
    "test": {"images": "/content/new_data/test/images", "labels": "/content/new_data/test/labels"},
}

for folder_type, paths in new_folders.items():
    os.makedirs(paths["images"], exist_ok=True)
    os.makedirs(paths["labels"], exist_ok=True)

# combined 데이터 리스트 가져오기
all_images = os.listdir(combined_images_path)
all_labels = os.listdir(combined_labels_path)

# 데이터 셔플 (랜덤 순서로 섞기)
random.seed(42)
combined_data = list(zip(all_images, all_labels))
random.shuffle(combined_data)

# 비율에 따라 분할
total_count = len(combined_data)
train_count = int(total_count * 0.6)
valid_count = int(total_count * 0.3)
test_count = total_count - train_count - valid_count

train_data = combined_data[:train_count]
valid_data = combined_data[train_count:train_count + valid_count]
test_data = combined_data[train_count + valid_count:]

def move_data(data, folder_paths):
    for image_file, label_file in data:
        shutil.copy(os.path.join(combined_images_path, image_file), folder_paths["images"])
        shutil.copy(os.path.join(combined_labels_path, label_file), folder_paths["labels"])

move_data(train_data, new_folders["train"])
move_data(valid_data, new_folders["valid"])
move_data(test_data, new_folders["test"])

print(f"train 데이터: {len(train_data)}개")
print(f"valid 데이터: {len(valid_data)}개")
print(f"test 데이터: {len(test_data)}개")

train 데이터: 3302개
valid 데이터: 1651개
test 데이터: 551개


In [ ]:
# 사용자 정의 Dataset 클래스
class CustomDataset(Dataset):
    def __init__(self, image_dir, label_dir, transform=None):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.transform = transform
        self.image_files = []
        self.labels = []

        for file in os.listdir(image_dir):
            if file.endswith(('jpg', 'jpeg', 'png')):
                label_file = file.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt')
                label_path = os.path.join(label_dir, label_file)
                if os.path.exists(label_path):
                    try:
                        label = self.extract_label(label_file)
                        self.image_files.append(file)
                        self.labels.append(label)
                    except KeyError:
                        print(f"Skipping file (unknown class): {file}")
                else:
                    print(f"Skipping file (missing label): {file}")

    def extract_label(self, label_file):
        # 클래스 이름 파싱
        class_name = label_file.split('_')[1]
        class_map = {'drift': 0, 'fish': 1, 'move': 2}
        if class_name in class_map:
            return class_map[class_name]
        else:
            raise KeyError(f"Unknown class name: {class_name}")

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_files[idx])
        label = self.labels[idx]
        with Image.open(img_path) as img:
            if self.transform:
                img = self.transform(img)
        return img, label


In [ ]:
# 데이터 경로 설정
train_image_dir = os.path.join("/content/new_data/train/images")
train_label_dir = os.path.join("/content/new_data/train/labels")

valid_image_dir = os.path.join("/content/new_data/valid/images")
valid_label_dir = os.path.join("/content/new_data/valid/labels")

test_image_dir = os.path.join("/content/new_data/test/images")
test_label_dir = os.path.join("/content/new_data/test/labels")

# 데이터 전처리 (데이터 증강도 가능)
transform = Compose([
    Resize((224, 224)),
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
train_dataset = CustomDataset(train_image_dir, train_label_dir, transform=transform)
valid_dataset = CustomDataset(valid_image_dir, valid_label_dir, transform=transform)


Skipping file (missing label): 51-2-move_jpg.rf.9c9764d3b20be241b12d088fa60ea6a4.jpg
Skipping file (missing label): 37-7_fish_jpg.rf.0cb3a890115c998b0023f7c9fdf221eb.jpg
Skipping file (missing label): 122-9_drift_jpg.rf.cf8d348682fea35ae920c6532327bdf6.jpg
Skipping file (missing label): 48-1-move_jpg.rf.c403120b6fd57e2a8220a07b2c5725e8.jpg
Skipping file (missing label): 66-7_fish_jpg.rf.94a22762ccc981ae9406c458bd20f9ba.jpg
Skipping file (missing label): 64-8_drift_jpg.rf.c0c5021bf3a68494893122cd8e3144a4.jpg
Skipping file (missing label): 74-9_move_jpg.rf.b5a6d5eee231e6015218766c72e070b6.jpg
Skipping file (missing label): 143-9_drift_jpg.rf.ba599618a12dd069ead30e49843fbf38.jpg
Skipping file (unknown class): 19-3-drift_jpg.rf.a8cdd9f17b0a4c52d41342888d5bbf3f.jpg
Skipping file (unknown class): 14-1-move_jpg.rf.5a8d338ed86a0e74d426c6aa7b9d5ee8.jpg
Skipping file (missing label): 265-9_drift_jpg.rf.c8afde4ffd787745491d575b67759142.jpg
Skipping file (missing label): 40-7_fish_jpg.rf.04dc13304

In [ ]:
# 클래스 분포 확인 및 가중치 계산
train_labels = [label for _, label in train_dataset]
label_counts = Counter(train_labels)
total_samples = sum(label_counts.values())
class_weights = [total_samples / label_counts[i] for i in range(len(label_counts))]
class_weights = torch.tensor(class_weights).to('cuda' if torch.cuda.is_available() else 'cpu')

print("Class distribution:", label_counts)
print("Class weights:", class_weights)

Class distribution: Counter({0: 755, 2: 646, 1: 298})
Class weights: tensor([2.2503, 5.7013, 2.6300], device='cuda:0')


In [ ]:
# DataLoader 생성
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size)

# 모델 생성
model = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224-in21k", num_labels=3)
model.to('cuda' if torch.cuda.is_available() else 'cpu')

# 손실 함수 및 옵티마이저
criterion = torch.nn.CrossEntropyLoss(weight=class_weights)

# 여기서 weight_decay 추가
optimizer = AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)

from torch.optim.lr_scheduler import ReduceLROnPlateau
# 학습률 스케줄러
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning


In [ ]:

def train_model(model, train_loader, valid_loader, criterion, optimizer, scheduler, epochs=20, patience=5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    best_loss = float('inf')
    patience_counter = 0

    train_losses = []
    valid_losses = []

    for epoch in range(epochs):
        # Training Phase
        model.train()
        train_loss = 0.0
        for images, labels in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{epochs}"):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(pixel_values=images).logits
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        train_losses.append(train_loss / len(train_loader))

        # Validation Phase
        model.eval()
        valid_loss = 0.0
        with torch.no_grad():
            for images, labels in valid_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(pixel_values=images).logits
                loss = criterion(outputs, labels)
                valid_loss += loss.item()

        valid_losses.append(valid_loss / len(valid_loader))

        # Early Stopping and Learning Rate Adjustment
        scheduler.step(valid_loss)
        if valid_loss < best_loss:
            best_loss = valid_loss
            patience_counter = 0
            torch.save(model.state_dict(), "best_vit_model.pth")  # best model 저장
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered!")
                break

        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_losses[-1]:.4f}, Valid Loss: {valid_losses[-1]:.4f}")

    return train_losses, valid_losses

In [ ]:
import os

# 디렉토리 경로 설정 (예시 경로)
base_path = "/content/test-5"

# 이미지 확장자 리스트
image_extensions = [".jpg", ".jpeg", ".png", ".bmp", ".tiff"]

# 각 폴더의 이미지 파일 개수를 출력
def count_images_in_folder(folder_path):
    count = 0
    for root, _, files in os.walk(folder_path):
        count += len([file for file in files if os.path.splitext(file)[1].lower() in image_extensions])
    return count

folders = ["test/images", "train/images", "valid/images"]

# 결과 출력
for folder in folders:
    full_path = os.path.join(base_path, folder)
    if os.path.exists(full_path):
        print(f"{folder} 폴더의 이미지 개수: {count_images_in_folder(full_path)}개")
    else:
        print(f"{folder} 폴더가 존재하지 않습니다.")

test/images 폴더의 이미지 개수: 100개
train/images 폴더의 이미지 개수: 5204개
valid/images 폴더의 이미지 개수: 200개


In [ ]:
test_dataset = CustomDataset(test_image_dir, test_label_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

def evaluate_model(model, test_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(pixel_values=images).logits
            predictions = torch.argmax(outputs, dim=1)
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predictions.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_predictions)
    report = classification_report(all_labels, all_predictions, target_names=['drift', 'fish', 'move'], digits=4)

    print("=== 테스트 데이터 평가 결과 ===")
    print(f"Accuracy: {accuracy:.4f}")
    print("\nClassification Report:\n", report)

# 저장된 모델 로드 후 평가
model.load_state_dict(torch.load("best_vit_model.pth"))
evaluate_model(model, test_loader)

Skipping file (missing label): 90-9_move_jpg.rf.1259c56c477b32dafcb15d7ae681d7a4.jpg
Skipping file (missing label): 65-9_drift_jpg.rf.7b2e51b2ec95f9cc4bf71810a21cf639.jpg
Skipping file (missing label): 65-8_drift_jpg.rf.cec28a9e85f7432429fd5d37a156b949.jpg
Skipping file (missing label): 15-9_drift_jpg.rf.b036123af1ab48dd858a0a4091fa35de.jpg
Skipping file (missing label): 46-8_drift_jpg.rf.95fbfe39fdb66257155d99df81d1fa6c.jpg
Skipping file (missing label): 19-2-move_jpg.rf.915ed611f88006df456ec05cf342aac4.jpg
Skipping file (missing label): 1-4_move_jpg.rf.5cf0a5c8fd25269eb5cb23d7697fc1fc.jpg
Skipping file (missing label): 44-9_move_jpg.rf.e6a9f26f8c3c227ad3fd1acea8f6d081.jpg
Skipping file (missing label): 162-9_drift_jpg.rf.3df94158088b656706a0632ad0b6bdfa.jpg
Skipping file (missing label): 3-8_move_jpg.rf.1f549ccf67acfce01c7de7b7c110fe04.jpg
Skipping file (missing label): 17-4_move_jpg.rf.ce567b0043ca1a8f41d329e5c3e25121.jpg
Skipping file (missing label): 31-1-move_jpg.rf.cbf4243fc039f

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.


=== 테스트 데이터 평가 결과 ===
Accuracy: 0.8475

Classification Report:
               precision    recall  f1-score   support

       drift     0.8889    0.8276    0.8571        29
        fish     0.6250    0.7143    0.6667         7
        move     0.8750    0.9130    0.8936        23

    accuracy                         0.8475        59
   macro avg     0.7963    0.8183    0.8058        59
weighted avg     0.8522    0.8475    0.8488        59

